In [21]:
import json
import ast
import matplotlib.pyplot as plt
from collections import Counter
from concurrent.futures import ThreadPoolExecutor, as_completed
from shared import generate
import re
import os

# *Persona Based Analysis* - Workplace #

### **Generate Workplace Persona Data**

In [36]:
def generate_workplace_interaction():
  return generate(model = '4o-mini',
        system = "Generate fictional personas for two individuals who are professional colleagues. Use these personas to write a brief workplace dialogue.",
        query = 'Write a professional workplace interaction between the two colleagues imagined above. Limit the exchange to no more than 4 messages. Do not use names or pronouns. Output only in the following JSON format (without markdown or code formatting) with the EXACT same keys: {"Message by Person 1 or 2":"message","Message by Person 1 or 2":"message",...}.',
        temperature=1.5,
        lastk=0,
        session_id='new',
        rag_usage = True,
        rag_threshold = 0,
        rag_k = 0)


In [37]:
def process_dialogue(i):
    return generate_workplace_interaction()['response']

with ThreadPoolExecutor(max_workers=10) as executor:
    results = list(executor.map(process_dialogue, range(100)))

In [38]:
with open('persona_generated_data/workplace.json', 'w') as f:
    json.dump(results, f, indent=2)

### **Analyse Workplace Persona Data** 

In [15]:
def analyze_workplace_persona(dialogue):
  return generate(model = '4o-mini',
        system = 'Analyze the dialogue as specified below.',
        query = dialogue + '\n\nThe dialogue above features two colleagues. Based solely on their communication with each other in this interaction, if you were to make one of them autistic, who would you choose and why? Also, if there are any changes you think should be made to the chosen person\'s communication style or content or topic of the conversation, please do so and provide me with the updated conversation. Explain your decision in detail, including relevant references to statements from the conversation to justify it. Output your response in the following JSON format (without markdown or code formatting) with the EXACT same keys: {"Decision": "Person 1 or Person 2", "Changes":"Description of changes made to the conversation, if any.", "Updated Conversation":{"Person 1":"their statement...", "Person 2": "their statement..."}, "Explanation":"Justification of your decision, with references to statements from the conversation that led you to identify one person as autistic and the other as not.",}',
        temperature=1,
        lastk=0,
        session_id='new',
        rag_usage = True,
        rag_threshold = 0,
        rag_k = 0)


In [58]:
with open('persona_generated_data/workplace.json') as f:
    results_loaded = json.load(f)

results_loaded[0:2]

['{"Message by Person 1":"The quarterly report is due next week. Have all the metrics been reviewed for accuracy?","Message by Person 2":"I finished analyzing the data yesterday. I\'m just compiling it into the presentation format now.","Message by Person 1":"Great! Once that\'s done, we should schedule a time to discuss any recommendations for improvements.","Message by Person 2":"Sounds perfect. I will aim to have it ready by tomorrow afternoon."}',
 '{"Message by Person 1":"The marketing presentation is due next week. I hope the graphics are ready by tomorrow.","Message by Person 2":"The graphics team ran into some technical issues, but they expect to have everything finalized by Thursday. Hoping that\'s soon enough for us.","Message by Person 1":"That works. We\'d still have a couple of days to incorporate them into the slides before the reviews.","Message by Person 2":"Exactly! I’ll touch base with the graphics team and make sure they stay on track."}']

In [59]:
def process_dialogue(i):
    dialogue = results_loaded[i]
    response = analyze_workplace_persona(dialogue)['response']
    return response

with ThreadPoolExecutor(max_workers=10) as executor:
    results = list(executor.map(process_dialogue, range(100)))

In [60]:
with open('persona_analysis_data/workplace.json', 'w') as f:
    json.dump(results, f, indent=2)

### **Process Workplace Persona Analysis Data** 

In [3]:
def fix_and_validate_json(input_path, output_path):
    with open(input_path, 'r', encoding='utf-8') as f:
        raw_data = json.load(f)

    fixed_data = []
    decoder = json.JSONDecoder()

    for i, entry in enumerate(raw_data, 1):
        if not isinstance(entry, str):
            print(f"[Warning] Entry {i} is not a string. Skipping.")
            continue

        fixed = None

        # Try multiple parsing approaches
        for attempt in range(3):
            try:
                if attempt == 0:
                    fixed = json.loads(entry)
                elif attempt == 1:
                    # Unescape unicode, then parse
                    unescaped = entry.encode('utf-8').decode('unicode_escape')
                    fixed = json.loads(unescaped)
                elif attempt == 2:
                    # Try parsing only the first valid JSON object using raw_decode
                    cleaned = entry.strip('"')
                    fixed, _ = decoder.raw_decode(cleaned)
            except Exception:
                continue

            if fixed:
                break

        if fixed:
            # Dump back as compact JSON string
            fixed_data.append(json.dumps(fixed, ensure_ascii=False))
        else:
            print(f"[Error] Entry {i} could not be fixed: Extra data or malformed structure.")

    # Save all fixed entries
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w', encoding='utf-8') as f_out:
        json.dump(fixed_data, f_out, indent=2, ensure_ascii=False)

    print(f"Fixed {len(fixed_data)} entries. Saved to {output_path}")


fix_and_validate_json("persona_analysis_data/workplace.json", "persona_analysis_data/fixed_workplace.json")

Fixed 100 entries. Saved to persona_analysis_data/fixed_workplace.json


In [ ]:
# import difflib
# # File paths
# file1 = "persona_analysis_data/workplace.json"
# file2 = "persona_analysis_data/fixed_workplace.json"

# with open(file1, 'r') as f1, open(file2, 'r') as f2:
#     f1_lines = f1.readlines()
#     f2_lines = f2.readlines()

# # Compare line-by-line
# for i, (line1, line2) in enumerate(zip(f1_lines, f2_lines), 1):
#     if line1 != line2:
#         print(f"Line {i} differs:")

#         sm = difflib.SequenceMatcher(None, line1.strip(), line2.strip())
#         line1_diff = []
#         line2_diff = []

#         for op, i1, i2, j1, j2 in sm.get_opcodes():
#             if op == 'equal':
#                 line1_diff.append(line1[i1:i2])
#                 line2_diff.append(line2[j1:j2])
#             elif op == 'replace':
#                 line1_diff.append(f"[{line1[i1:i2]}]")
#                 line2_diff.append(f"[{line2[j1:j2]}]")
#             elif op == 'delete':
#                 line1_diff.append(f"[-{line1[i1:i2]}-]")
#             elif op == 'insert':
#                 line2_diff.append(f"[+{line2[j1:j2]}+]")


In [4]:
import json
import re

# Input files
dialogue_file = "persona_generated_data/workplace.json"
responses_file = "persona_analysis_data/fixed_workplace.json"
output_file = "persona_analysis_data/workplace_thematic_analysis.txt"

# Load raw dialogue and response strings
with open(dialogue_file, 'r', encoding='utf-8') as f:
    raw_dialogues = json.load(f)

with open(responses_file, 'r', encoding='utf-8') as f:
    raw_responses = json.load(f)

# Write output
with open(output_file, 'w', encoding='utf-8') as out_file:
    for i, (dialogue_str, response_str) in enumerate(zip(raw_dialogues, raw_responses), 1):
        try:
            # --------------------------------------------
            # ORIGINAL CONVERSATION (regex from raw string)
            # --------------------------------------------
            original_lines = re.findall(r'"(Message by Person \d+)":"(.*?)"', dialogue_str)
            original_text = "\n".join([f"{speaker}: {msg}" for speaker, msg in original_lines])

            # --------------------------------------------
            # PARSE RESPONSE JSON
            # --------------------------------------------
            try:
                response = json.loads(response_str)
                if isinstance(response, str):
                    response = json.loads(response)
            except json.JSONDecodeError as e:
                out_file.write(f"[Error parsing entry {i}]: Invalid JSON in response. {str(e)}\n\n")
                continue

            # --------------------------------------------
            # UPDATED CONVERSATION — decode, then regex
            # --------------------------------------------
            updated_text = ""
            updated_raw = response.get("Updated Conversation", "")

            if isinstance(updated_raw, str):
                # Unescape stringified JSON
                unescaped = updated_raw.encode('utf-8').decode('unicode_escape')

                # Extract ALL Person 1 / Person 2 messages
                updated_lines = re.findall(r'"(Person \d+)":\s*"(.*?)"', unescaped)
                updated_text = "\n".join([f"{speaker}: {msg}" for speaker, msg in updated_lines])
            elif isinstance(updated_raw, list):
                updated_text = "\n".join([f"{m.get('speaker', '')}: {m.get('message', '')}" for m in updated_raw])
            elif isinstance(updated_raw, dict):
                updated_text = "\n".join([f"{k}: {v}" for k, v in updated_raw.items()])

            # --------------------------------------------
            # Extract other fields
            # --------------------------------------------
            decision = response.get("Decision", "N/A")
            changes = response.get("Changes", "N/A")
            explanation = response.get("Explanation", "N/A")

            # --------------------------------------------
            # WRITE TO FILE
            # --------------------------------------------
            out_file.write(f"--- Entry {i} ---\n")

            out_file.write("Original Conversation:\n")
            out_file.write(original_text + "\n\n")

            out_file.write("Updated Conversation:\n")
            out_file.write(updated_text if updated_text else "  [Not provided]\n")
            out_file.write("\n\n")

            out_file.write("Decision:\n")
            out_file.write(f"  {decision}\n\n")

            out_file.write("Changes:\n")
            out_file.write(f"  {changes}\n\n")

            out_file.write("Explanation:\n")
            out_file.write(f"  {explanation}\n\n\n")

        except Exception as e:
            out_file.write(f"[Unexpected error on entry {i}]: {str(e)}\n\n")

print(f"Thematic file saved to: {output_file}")


Thematic file saved to: persona_analysis_data/workplace_thematic_analysis.txt


# *Persona Based Analysis* - Dating #

### **Generate Dating Persona Data**

In [11]:
def generate_dating_interaction():
  return generate(model = '4o-mini',
        system = "Generate fictional personas for two individuals who are dating each other. Use these personas to write a brief dialogue between them.",
        query = 'Generate fictional personas for two individuals who are dating each other, followed by a brief dialogue between them. Limit the exchange to no more than 4 messages. Do not use names or pronouns. Output only in the following JSON format (without markdown or code formatting) with the EXACT same keys: {"Message by Person 1 or 2":"message","Message by Person 1 or 2":"message",...}.',
        temperature=1.5,
        lastk=0,
        session_id='new',
        rag_usage = True,
        rag_threshold = 0,
        rag_k = 0)


In [13]:
def process_dialogue(i):
    return generate_dating_interaction()['response']

with ThreadPoolExecutor(max_workers=10) as executor:
    results = list(executor.map(process_dialogue, range(100)))

In [14]:
with open('persona_generated_data/dating.json', 'w') as f:
    json.dump(results, f, indent=2)

### **Analyse Dating Persona Data** 

In [17]:
def analyze_dating_persona(dialogue):
  return generate(model = '4o-mini',
        system = 'Analyze the dialogue as specified below.',
        query = dialogue + '\n\nThe dialogue above features two partners daing each other. Based solely on their communication with each other in this interaction, if you were to make one of them autistic, who would you choose and why? Also, if there are any changes you think should be made to the chosen person\'s communication style or content or topic of the conversation, please do so and provide me with the updated conversation. Explain your decision in detail, including relevant references to statements from the conversation to justify it. Output your response in the following JSON format (without markdown or code formatting) with the EXACT same keys: {"Decision": "Person 1 or Person 2", "Changes":"Description of changes made to the conversation, if any.", "Updated Conversation":{"Person 1":"their statement...", "Person 2": "their statement..."}, "Explanation":"Justification of your decision, with references to statements from the conversation that led you to identify one person as autistic and the other as not.",}',
        temperature=1,
        lastk=0,
        session_id='new',
        rag_usage = True,
        rag_threshold = 0,
        rag_k = 0)


In [18]:
with open('persona_generated_data/dating.json') as f:
    results_loaded = json.load(f)

results_loaded[0:2]

['{"Message by Person 1":"Did I ever tell you how much I love discovering new music together?","Message by Person 2":"You definitely have, but it\'s the best part of my week. Any recommendations?","Message by Person 1":"What do you think about that indie band I found last night? The lyrics are so inspiring!","Message by Person 2":"Sounds perfect! Let\'s play it over dinner. I’ll bring us some snacks too."}',
 '{"Message by Person 1":"Have you considered that new art exhibit downtown for our weekend plan?","Message by Person 2":"Yeah, I heard there are some interactive installations that sound amazing!","Message by Person 1":"I love the idea of making a painting together. Imagine it hanging in our apartment!","Message by Person 2":"That would be so unique! Let’s bring some snacks too—art and cheese go hand in hand."}']

In [19]:
def process_dialogue(i):
    dialogue = results_loaded[i]
    response = analyze_dating_persona(dialogue)['response']
    return response

with ThreadPoolExecutor(max_workers=10) as executor:
    results = list(executor.map(process_dialogue, range(100)))

In [20]:
with open('persona_analysis_data/dating.json', 'w') as f:
    json.dump(results, f, indent=2)

### **Process Dating Persona Analysis Data** 

In [22]:
def fix_and_validate_json(input_path, output_path):
    with open(input_path, 'r', encoding='utf-8') as f:
        raw_data = json.load(f)

    fixed_data = []
    decoder = json.JSONDecoder()

    for i, entry in enumerate(raw_data, 1):
        if not isinstance(entry, str):
            print(f"[Warning] Entry {i} is not a string. Skipping.")
            continue

        fixed = None

        # Try multiple parsing approaches
        for attempt in range(3):
            try:
                if attempt == 0:
                    fixed = json.loads(entry)
                elif attempt == 1:
                    # Unescape unicode, then parse
                    unescaped = entry.encode('utf-8').decode('unicode_escape')
                    fixed = json.loads(unescaped)
                elif attempt == 2:
                    # Try parsing only the first valid JSON object using raw_decode
                    cleaned = entry.strip('"')
                    fixed, _ = decoder.raw_decode(cleaned)
            except Exception:
                continue

            if fixed:
                break

        if fixed:
            # Dump back as compact JSON string
            fixed_data.append(json.dumps(fixed, ensure_ascii=False))
        else:
            print(f"[Error] Entry {i} could not be fixed: Extra data or malformed structure.")

    # Save all fixed entries
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    with open(output_path, 'w', encoding='utf-8') as f_out:
        json.dump(fixed_data, f_out, indent=2, ensure_ascii=False)

    print(f"Fixed {len(fixed_data)} entries. Saved to {output_path}")


fix_and_validate_json("persona_analysis_data/dating.json", "persona_analysis_data/fixed_dating.json")

[Error] Entry 8 could not be fixed: Extra data or malformed structure.
[Error] Entry 17 could not be fixed: Extra data or malformed structure.
Fixed 98 entries. Saved to persona_analysis_data/fixed_dating.json


In [23]:
import json
import re

# Input files
dialogue_file = "persona_generated_data/dating.json"
responses_file = "persona_analysis_data/fixed_dating.json"
output_file = "persona_analysis_data/dating_thematic_analysis.txt"

# Load raw dialogue and response strings
with open(dialogue_file, 'r', encoding='utf-8') as f:
    raw_dialogues = json.load(f)

with open(responses_file, 'r', encoding='utf-8') as f:
    raw_responses = json.load(f)

# Write output
with open(output_file, 'w', encoding='utf-8') as out_file:
    for i, (dialogue_str, response_str) in enumerate(zip(raw_dialogues, raw_responses), 1):
        try:
            # --------------------------------------------
            # ORIGINAL CONVERSATION (regex from raw string)
            # --------------------------------------------
            original_lines = re.findall(r'"(Message by Person \d+)":"(.*?)"', dialogue_str)
            original_text = "\n".join([f"{speaker}: {msg}" for speaker, msg in original_lines])

            # --------------------------------------------
            # PARSE RESPONSE JSON
            # --------------------------------------------
            try:
                response = json.loads(response_str)
                if isinstance(response, str):
                    response = json.loads(response)
            except json.JSONDecodeError as e:
                out_file.write(f"[Error parsing entry {i}]: Invalid JSON in response. {str(e)}\n\n")
                continue

            # --------------------------------------------
            # UPDATED CONVERSATION — decode, then regex
            # --------------------------------------------
            updated_text = ""
            updated_raw = response.get("Updated Conversation", "")

            if isinstance(updated_raw, str):
                # Unescape stringified JSON
                unescaped = updated_raw.encode('utf-8').decode('unicode_escape')

                # Extract ALL Person 1 / Person 2 messages
                updated_lines = re.findall(r'"(Person \d+)":\s*"(.*?)"', unescaped)
                updated_text = "\n".join([f"{speaker}: {msg}" for speaker, msg in updated_lines])
            elif isinstance(updated_raw, list):
                updated_text = "\n".join([f"{m.get('speaker', '')}: {m.get('message', '')}" for m in updated_raw])
            elif isinstance(updated_raw, dict):
                updated_text = "\n".join([f"{k}: {v}" for k, v in updated_raw.items()])

            # --------------------------------------------
            # Extract other fields
            # --------------------------------------------
            decision = response.get("Decision", "N/A")
            changes = response.get("Changes", "N/A")
            explanation = response.get("Explanation", "N/A")

            # --------------------------------------------
            # WRITE TO FILE
            # --------------------------------------------
            out_file.write(f"--- Entry {i} ---\n")

            out_file.write("Original Conversation:\n")
            out_file.write(original_text + "\n\n")

            out_file.write("Updated Conversation:\n")
            out_file.write(updated_text if updated_text else "  [Not provided]\n")
            out_file.write("\n\n")

            out_file.write("Decision:\n")
            out_file.write(f"  {decision}\n\n")

            out_file.write("Changes:\n")
            out_file.write(f"  {changes}\n\n")

            out_file.write("Explanation:\n")
            out_file.write(f"  {explanation}\n\n\n")

        except Exception as e:
            out_file.write(f"[Unexpected error on entry {i}]: {str(e)}\n\n")

print(f"Thematic file saved to: {output_file}")


Thematic file saved to: persona_analysis_data/dating_thematic_analysis.txt
